# **¿Cuál es la mejor tarifa?**
Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

# **Inicialización**

In [ ]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math

# **Cargar datos**

In [ ]:
# Carga los archivos de datos en diferentes DataFrames
calls = pd.read_csv('/datasets/megaline_calls.csv')
internet = pd.read_csv('/datasets/megaline_internet.csv')
messages = pd.read_csv('/datasets/megaline_messages.csv')
plans = pd.read_csv('/datasets/megaline_plans.csv')
users = pd.read_csv('/datasets/megaline_users.csv')



# **Preparar los datos**

**1.4. Tarifas**

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de las tarifas
print (plans.info())


In [ ]:
# Imprime una muestra de los datos para las tarifas
print(plans.head())


**Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]**

No hay valores nulos en el DataFrame de tarifas.

Los tipos de datos son adecuados.

Los nombres de columna podrían mejorarse para facilitar el uso.

**1.5. Corregir datos**

In [ ]:
plans.columns = [
    'messages_included', 'mb_per_month_included', 'minutes_included',
    'usd_monthly_fee', 'usd_per_gb', 'usd_per_message',
    'usd_per_minute', 'plan_name'
]


**1.6. Enriquecer los datos**

In [ ]:
plans['gb_per_month_included'] = plans['mb_per_month_included'] / 1024

**1.7. Usuarios/as**

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de usuarios

print (users.info())


In [ ]:
# Imprime una muestra de datos para usuarios
print (users.head())

**[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]**

Solo 34 de 500 usuarios han cancelado su suscripción (columna churn_date).

reg_date y churn_date están como objetos y deberían convertirse en fechas.

**1.7.1. Corregir los datos**

In [ ]:
users['reg_date'] = pd.to_datetime(users['reg_date'])
users['churn_date'] = pd.to_datetime(users['churn_date'])


**1.7.2. Enriquecer los datos**

In [ ]:
users['reg_month'] = users['reg_date'].dt.month

**1.8. Llamadas**

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas
print (calls.info())

In [ ]:
# Imprime una muestra de datos para las llamadas
print (calls.head())

**[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]**

El DataFrame tiene 137,735 registros y 4 columnas. No hay valores nulos. El campo call_date aparece como tipo object, pero debería ser datetime. La duración de las llamadas está en float, pero Megaline redondea cada llamada individual hacia arriba al minuto siguiente.

**Cómo se puede arreglar? **Convertir call_date a tipo datetime. Aplicar redondeo hacia arriba (np.ceil()) a la columna duration. Agregar la columna month para análisis mensuales posteriores.

**1.8.1. Corregir los datos**

In [ ]:
calls['call_date'] = pd.to_datetime(calls['call_date'])
calls['duration'] = np.ceil(calls['duration'])

**1.8.2. Enriquecer los datos**

In [ ]:
calls['month'] = calls['call_date'].dt.month


**1.9. Mensajes**

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes
print(messages.info())

In [ ]:
# Imprime una muestra de datos para los mensajes
print(messages.head())

**[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]**

El DataFrame tiene 76,051 filas y 3 columnas. No hay valores nulos. La columna message_date aparece como tipo object, pero debe ser de tipo datetime para análisis temporales. No hay columnas adicionales que indiquen el mes.

**1.9.1. Corregir los datos**

In [ ]:
messages['message_date'] = pd.to_datetime(messages['message_date'])


**1.9.2. Enriquecer los datos**

In [ ]:
messages['month'] = messages['message_date'].dt.month

**1.10. Internet**

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de internet
print(internet.info())


In [ ]:
# Imprime una muestra de datos para el tráfico de internet
print(internet.head())

**[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]**

El DataFrame contiene 104,825 registros con 4 columnas. No hay valores nulos. La columna session_date es de tipo object, pero debería ser tipo datetime. El tráfico de datos (mb_used) está expresado en megabytes, y según las reglas de negocio, se debe redondear el total mensual a GB hacia arriba. No hay una columna month, la cual es útil para agrupar por mes.

**1.10.1. Corregir los datos**

In [ ]:
internet['session_date'] = pd.to_datetime(internet['session_date'])

**1.10.2. Enriquecer los datos**

In [ ]:
internet['month'] = internet['session_date'].dt.month

**1.11. Estudiar las condiciones de las tarifas**

In [ ]:
# Imprime las condiciones de la tarifa y asegúrate de que te quedan claras
print(plans.info())
print(plans.head())

**1.12. Agregar datos por usuario**

In [ ]:
# Calcula el número de llamadas hechas por cada usuario al mes. Guarda el resultado.
calls_per_month = calls.groupby(['user_id', 'month'])['id'].count().reset_index().rename(columns={'id': 'calls'})


In [ ]:
# Calcula la cantidad de minutos usados por cada usuario al mes. Guarda el resultado.
minutes_per_month = calls.groupby(['user_id', 'month'])['duration'].sum().reset_index()

In [ ]:
# Calcula el número de mensajes enviados por cada usuario al mes. Guarda el resultado.
messages_per_month = messages.groupby(['user_id', 'month'])['id'].count().reset_index().rename(columns={'id': 'messages'})

In [ ]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.
internet_per_month = internet.groupby(['user_id', 'month'])['mb_used'].sum().reset_index()


In [ ]:
# Fusiona los datos de llamadas, minutos, mensajes e Internet con base en user_id y month

monthly_data = minutes_per_month.copy()
monthly_data = monthly_data.merge(calls_per_month, on=['user_id', 'month'], how='left')
monthly_data = monthly_data.merge(messages_per_month, on=['user_id', 'month'], how='left')
monthly_data = monthly_data.merge(internet_per_month, on=['user_id', 'month'], how='left')
monthly_data = monthly_data.fillna(0)

In [ ]:
# Añade la información de la tarifa
monthly_data = monthly_data.merge(users[['user_id', 'plan']], on='user_id', how='left')
monthly_data = monthly_data.merge(plans, left_on='plan', right_on='plan_name', how='left')

In [ ]:
# Calcula el ingreso mensual para cada usuario
monthly_data['gb_used'] = np.ceil(monthly_data['mb_used'] / 1024)

monthly_data['extra_minutes'] = (monthly_data['duration'] - monthly_data['minutes_included']).clip(lower=0)
monthly_data['extra_messages'] = (monthly_data['messages'] - monthly_data['messages_included']).clip(lower=0)
monthly_data['extra_gb'] = (monthly_data['gb_used'] - (monthly_data['mb_per_month_included'] / 1024)).clip(lower=0)

monthly_data['extra_cost'] = (monthly_data['extra_minutes'] * monthly_data['usd_per_minute'] +
                              monthly_data['extra_messages'] * monthly_data['usd_per_message'] +
                              monthly_data['extra_gb'] * monthly_data['usd_per_gb'])

monthly_data['monthly_revenue'] = monthly_data['usd_monthly_fee'] + monthly_data['extra_cost']

In [ ]:
monthly_data.columns

**1.13. Estudia el comportamiento de usuario**

**1.13.1. Llamadas**

In [ ]:
# Compara la duración promedio de llamadas por cada plan y por cada mes. Traza un gráfico de barras para visualizarla.

call_stats = monthly_data.groupby(['plan', 'month'])['duration'].mean()
print(call_stats)


call_stats.T.plot(kind='bar', figsize=(12, 6))
plt.title('Duración promedio de llamadas por mes y plan')
plt.xlabel('Mes')
plt.ylabel('Duración promedio (minutos)')
plt.xticks(rotation=0)
plt.legend(title='Plan')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
#Calcula la duracion promedio de cada plan y cada mes
avg_duration = monthly_data.groupby(['plan', 'month'])['duration'].mean().reset_index()

# Creacion de grafico de barras
plt.figure(figsize=(14, 6))
sns.barplot(data=avg_duration, x='month', y='duration', hue='plan')
plt.title('Duracion promedio de llamadas por plan y mes')
plt.xlabel('Mes')
plt.ylabel('Duracion promedio de llamadas (minutos)')
plt.xticks(rotation=45)
plt.legend(title='Plan')
plt.tight_layout()
plt.show()

In [ ]:
# Compara el número de minutos mensuales que necesitan los usuarios de cada plan. Traza un histograma.

plt.figure(figsize=(12, 6))
for plan in monthly_data['plan'].unique():
    subset = monthly_data[monthly_data['plan'] == plan]
    sns.histplot(subset['duration'], kde=False, label=plan, bins=30, alpha=0.6)

plt.title('Distribución de minutos mensuales usados por plan')
plt.xlabel('Minutos')
plt.ylabel('Frecuencia')
plt.legend(title='Plan')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.

mean_duration = monthly_data.groupby('plan')['duration'].mean()
print("Media de duración mensual de llamadas por plan:")
print(mean_duration)

var_duration = monthly_data.groupby('plan')['duration'].var()
print("\nVarianza de duración mensual de llamadas por plan:")
print(var_duration)


In [ ]:
# Traza un diagrama de caja para visualizar la distribución de la duración mensual de llamadas

# Diagrama de caja de duración mensual por plan
plt.figure(figsize=(10, 6))
sns.boxplot(x='plan', y='duration', data=monthly_data)
plt.title('Distribución de duración mensual de llamadas por plan')
plt.xlabel('Plan')
plt.ylabel('Duración mensual (minutos)')
plt.show()


**1.13.2. Mensajes**

In [ ]:
# Comprara el número de mensajes que tienden a enviar cada mes los usuarios de cada plan

plt.figure(figsize=(12, 6))
for plan in monthly_data['plan'].unique():
    subset = monthly_data[monthly_data['plan'] == plan]
    sns.histplot(subset['duration'], kde=False, label=plan, bins=30, alpha=0.6)

(plt.title('Distribución de minutos mensuales usados por plan'),
 plt.xlabel('Minutos'),
 plt.ylabel('Frecuencia'),
 plt.legend(title='Plan'),
 plt.grid(True),
 plt.tight_layout(),
 plt.show())


In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan

plt.figure(figsize=(12, 6))
for plan in monthly_data['plan'].unique():
    sns.histplot(
        monthly_data[monthly_data['plan'] == plan]['gb_used'],
        label=plan,
        bins=30,
        kde=False,
        alpha=0.6
    )

(plt.title('Distribución de tráfico de Internet (GB) mensual por plan'),
 plt.xlabel('GB utilizados'),
 plt.ylabel('Frecuencia'),
 plt.legend(title='Plan'),
 plt.show())



**[Elabora las conclusiones sobre el comportamiento de los usuarios con respecto a los mensajes. ¿Su comportamiento varía en función del plan?]**

Los usuarios del plan Ultimate tienden a enviar más mensajes mensuales en comparación con los de Surf. En cuanto al tráfico de Internet, ambos grupos presentan un uso intensivo, pero el plan Ultimate permite más GB sin costo adicional, por lo que muchos usuarios probablemente prefieren esta tarifa.

**1.13.3. Internet**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12, 6))
for plan in monthly_data['plan'].unique():
    sns.histplot(
        data=monthly_data[monthly_data['plan'] == plan],
        x='gb_used',
        label=plan,
        bins=30,
        kde=False,
        alpha=0.6
    )

In [ ]:
plt.figure(figsize=(12, 6))
for plan in monthly_data['plan'].unique():
    sns.histplot(
        monthly_data[monthly_data['plan'] == plan]['messages'],
        label=plan,
        bins=30,
        kde=False,
        alpha=0.6
    )


**[Elabora las conclusiones sobre cómo los usuarios tienden a consumir el tráfico de Internet. ¿Su comportamiento varía en función del plan?]**

los usuarios con el plan Ultimate tienen un comportamiento más intensivo y relajado en cuanto al uso de Internet, probablemente porque su plan les ofrece más GB y menor penalización por exceso. En cambio, los usuarios de Surf parecen moderar su consumo.

**1.14. Ingreso**

In [ ]:
income_stats = monthly_data.groupby('plan')['monthly_revenue'].agg(['mean', 'median', 'var', 'std', 'min', 'max', 'count'])
print(income_stats)

In [ ]:

(
 plt.figure(figsize=(12, 6)),
 [sns.histplot(
     data=monthly_data[monthly_data['plan'] == plan],
     x='monthly_revenue',
     label=plan,
     bins=30,
     kde=True,
     alpha=0.6
)

In [ ]:
(
 plt.figure(figsize=(12, 6)),
 [sns.histplot(
     data=monthly_data[monthly_data['plan'] == plan],
     x='monthly_revenue',
     label=plan,
     bins=30,
     kde=True,
     alpha=0.6
 ) for plan in monthly_data['plan'].unique()],
 plt.title('Distribución de ingresos mensuales por plan'),
 plt.xlabel('Ingresos ($)'),
 plt.ylabel('Frecuencia'),
 plt.legend(title='Plan'),
 plt.grid(True),
 plt.tight_layout(),
 plt.show()
)


**[Elabora las conclusiones sobre cómo difiere el ingreso entre los planes.]**

Ultimate es más rentable por usuario y con menos variabilidad en los ingresos. Esto puede hacerlo más conveniente para campañas de retención o fidelización.

**1.15. Prueba las hipótesis estadísticas**

**[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]**

**Hipótesis nula (H₀):** Los ingresos promedio de los usuarios de los planes Surf y Ultimate no son diferentes.** Hipótesis alternativa (H₁)**: Los ingresos promedio de los usuarios de los planes Surf y Ultimate son diferentes. Valor alfa (nivel de significancia): 0.05

In [ ]:
# Prueba las hipótesis

from scipy import stats


surf_revenue = monthly_data[monthly_data['plan'] == 'surf']['monthly_revenue']
ultimate_revenue = monthly_data[monthly_data['plan'] == 'ultimate']['monthly_revenue']

t_stat, p_value = stats.ttest_ind(surf_revenue, ultimate_revenue, equal_var=False)  # Welch's t-test

print("Estadístico t:", t_stat)
print("Valor p:", p_value)

if p_value < 0.05:
    print("Rechazamos la hipótesis nula: hay diferencia significativa entre los ingresos promedio de Surf y Ultimate.")
else:
    print("No se rechaza la hipótesis nula: no hay diferencia significativa entre los ingresos promedio.")


**[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]**

**Hipótesis nula (H₀)**: Los ingresos promedio de los usuarios del área NY-NJ no difieren de los de otras regiones. **Hipótesis alternativa (H₁)**: Los ingresos promedio de los usuarios del área NY-NJ sí difieren de los de otras regiones. Valor alfa: 0.05

In [ ]:
# Prueba las hipótesis

monthly_data = monthly_data.merge(users[['user_id', 'city']], on='user_id', how='left')

ny_nj_revenue = monthly_data[monthly_data['city'].str.contains('New York|Jersey', case=False)]['monthly_revenue']
other_revenue = monthly_data[~monthly_data['city'].str.contains('New York|Jersey', case=False)]['monthly_revenue']

t_stat_ny, p_value_ny = stats.ttest_ind(ny_nj_revenue, other_revenue, equal_var=False)

print("Estadístico t (NY vs otros):", t_stat_ny)
print("Valor p (NY vs otros):", p_value_ny)

if p_value_ny < 0.05:
    print("Rechazamos la hipótesis nula: los ingresos promedio de NY/NJ difieren de otras regiones.")
else:
    print("No se rechaza la hipótesis nula: no hay diferencia significativa entre los ingresos promedio.")


# **Conclusión general**
**[En esta sección final, enumera tus conclusiones importantes. Asegúrate de que estas abarquen todas las decisiones (suposiciones) importantes que adoptaste y que determinaron la forma elegida para procesar y analizar los datos.]**

El plan Ultimate es más rentable por usuario, aunque su adopción puede ser limitada por su alto precio. El plan Surf puede ser mejor para atraer más usuarios, pero genera ingresos adicionales solo cuando se exceden los límites.

Recomendación: considerar una estrategia mixta, apuntando campañas de adquisición para Surf y de fidelización para Ultimate.